在训练我们的模型时，我们经常希望能够同时处理整个小批量的样本。 为了实现这一点，需要我们对计算进行矢量化， 从而利用线性代数库，而不是在Python中编写开销高昂的for循环。


In [9]:
from d2l.torch import d2l

from utils.tool import Timer
import torch

为了说明矢量化为什么如此重要，我们考虑对向量相加的两种方法。 我们实例化两个全为1的10000维向量。 在一种方法中，我们将使用Python的for循环遍历向量； 在另一种方法中，我们将依赖对+的调用。

In [10]:
n = 10000
a = torch.ones([n])
b = torch.ones([n])

现在我们可以对工作负载进行基准测试。

首先，我们使用for循环，每次执行一位的加法。

In [29]:
import time
import torch
class Timer:
    """记录多次运行时间"""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """启动计时器"""
        self.tik = time.time()

    def stop(self):
        """停止计时器并将时间记录在列表中"""
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        """返回平均时间"""
        return sum(self.times) / len(self.times)

    def sum(self):
        """返回时间总和"""
        return sum(self.times)

    def cumsum(self):
        """返回累计时间"""
        # 使用 PyTorch 的 cumsum 方法
        return torch.tensor(self.times).cumsum(dim=0).tolist()

n = 10000
c = torch.zeros(n)
timer = Timer()
for i in range(n):
    c[i] = a[i] + b[i]
f'{timer.stop():.5f} sec'

'0.14800 sec'

In [31]:
timer.start()
d = a + b
print(d)
f'{timer.stop():.5f} sec'

tensor([2., 2., 2.,  ..., 2., 2., 2.])


'0.00101 sec'

## 3.2. 线性回归的从零开始实现
在了解线性回归的关键思想之后，我们可以开始通过代码来动手实现线性回归了。 在这一节中，我们将从零开始实现整个方法， 包括数据流水线、模型、损失函数和小批量随机梯度下降优化器。 虽然现代的深度学习框架几乎可以自动化地进行所有这些工作，但从零开始实现可以确保我们真正知道自己在做什么。 同时，了解更细致的工作原理将方便我们自定义模型、自定义层或自定义损失函数。 在这一节中，我们将只使用张量和自动求导。 在之后的章节中，我们会充分利用深度学习框架的优势，介绍更简洁的实现方式。

In [53]:
#%matplotlib inline # 用于在 Jupyter Notebook 或其他支持 IPython 的环境中配置 Matplotlib 的显示方式为内联显示图形，不会弹出单独的窗口
import torch
#from d2l import torch as d2l

def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

注意，features中的每一行都包含一个二维数据样本， labels中的每一行都包含一维标签值（一个标量）。

In [55]:
print('features:', features[0],'\nlabel:', labels[0])

features: tensor([1.2873, 0.5567]) 
label: tensor([4.8926])
